# Build model

## import liraries

In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, classification_report
from pathlib import Path
from PIL import Image 
from keras import models
import os,random

print("Libraries imported")

Libraries imported


## Load data


In [4]:
dataPath = "/content/drive/MyDrive/Academic/7th Sem/Computer Vision/Mini project/data set/Digits"

data = os.listdir(dataPath)
dataX = []
dataY = []

dataClasses = len(data)

for i in range (1,dataClasses):
  data_list = os.listdir(dataPath +"/"+str(i))
  for j in data_list:
    pic = cv2.imread(dataPath +"/"+str(i)+"/"+j)
    pic = cv2.resize(pic,(28,28))
    dataX.append(pic)
    dataY.append(i)

dataX = np.array(dataX)
dataY = np.array(dataY)

## split data

In [5]:
trainX, testX, trainY, testY = train_test_split(dataX,dataY,test_size=0.1)
trainX, validX, trainY, validY = train_test_split(trainX,trainY,test_size=0.2)
print("Training Set Shape = ",trainX.shape)
print("Validation Set Shape = ",validX.shape)
print("Test Set Shape = ",testX.shape)

Training Set Shape =  (3076, 28, 28, 3)
Validation Set Shape =  (770, 28, 28, 3)
Test Set Shape =  (428, 28, 28, 3)


## preprocess data

In [ ]:
def Prep(img):
  #making image grayscale
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 

  #Histogram equalization to enhance contrast
  img = cv2.equalizeHist(img) 
  img = img/255 #normalizing
  return img

In [ ]:
trainX = np.array(list(map(Prep, trainX)))
testX = np.array(list(map(Prep, testX)))
validX= np.array(list(map(Prep, validX)))

## Reshape images

In [ ]:
trainX = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2],1)
testX = testX.reshape(testX.shape[0], testX.shape[1], testX.shape[2],1)
validX = validX.reshape(validX.shape[0], validX.shape[1], validX.shape[2],1)


## Augment data

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(trainX)

## generate labels

In [ ]:
trainY = to_categorical(trainY, dataClasses)
testY = to_categorical(testY, dataClasses)
validY = to_categorical(validY, dataClasses)

## NN (Dense net) model

In [ ]:
model = Sequential()

model.add((Conv2D(60,(5,5),input_shape=(32, 32, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


optimizer = RMSprop(learning_rate=0.005, rho=0.9, epsilon = 1e-08, decay=0.0)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#Fit the model



# Train model

In [ ]:
history = model.fit(datagen.flow(trainX, trainY, batch_size=100),
                              epochs = 30, validation_data = (validX, validY),
                              verbose = 2)
score = model.evaluate(testX, testY, verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

Epoch 1/30
39/39 - 60s - loss: 0.2958 - accuracy: 0.9039 - val_loss: 0.0425 - val_accuracy: 0.9843
Epoch 2/30
39/39 - 61s - loss: 0.2251 - accuracy: 0.9304 - val_loss: 0.0563 - val_accuracy: 0.9790
Epoch 3/30
39/39 - 61s - loss: 0.2207 - accuracy: 0.9343 - val_loss: 0.0256 - val_accuracy: 0.9906
Epoch 4/30
39/39 - 61s - loss: 0.2584 - accuracy: 0.9367 - val_loss: 0.0998 - val_accuracy: 0.9622
Epoch 5/30
39/39 - 61s - loss: 0.1158 - accuracy: 0.9638 - val_loss: 0.0494 - val_accuracy: 0.9822
Epoch 6/30
39/39 - 61s - loss: 0.1944 - accuracy: 0.9527 - val_loss: 0.0183 - val_accuracy: 0.9927
Epoch 7/30
39/39 - 61s - loss: 0.1174 - accuracy: 0.9735 - val_loss: 0.0231 - val_accuracy: 0.9916
Epoch 8/30
39/39 - 61s - loss: 0.1190 - accuracy: 0.9698 - val_loss: 0.0416 - val_accuracy: 0.9916
Epoch 9/30
39/39 - 61s - loss: 0.1177 - accuracy: 0.9730 - val_loss: 0.0229 - val_accuracy: 0.9916
Epoch 10/30
39/39 - 61s - loss: 0.0976 - accuracy: 0.9737 - val_loss: 0.0178 - val_accuracy: 0.9937
Epoch 11/

# save model

In [ ]:
# Save model in h5 format

model.save("/content/drive/MyDrive/Academic/7th Sem/Computer Vision/Mini project/model for digits/digit_classifier.h5")
print("Saved H5 model to drive")

Saved H5 model to drive
Saved H5 model to drive
